# Refinitiv Data Library for Python
## Content - Pricing - Chain + Streaming Constituents

This notebook demonstrates how to use the Pricing interface to 
- retrieve the constituents of a Chain instrument
- and then request Streaming data for the Constituents themselves

## Import the library and load credentials

Credentials used by this and the other tutorials notebooks are stored in the **Configuration/credentials.ipynb** file.     

You should have edited the **Configuration/credentials.ipynb** to set your credentials as part of the **Quick Start** step.

In [1]:
from refinitiv.data.content import pricing
from pandas import DataFrame
from IPython.display import display, clear_output

%run ../../Configuration/credentials.ipynb

## Open the session of your choice

Use our helper function in the Credentials notebook, **open_session(session_type)**, to create and open a session to connect to the 
- Refinitiv Data Platform directly (session_type="rdp") or via 
- Eikon 4 or Refinitiv Workspace (session_type="desktop") or via a 
- local realtime infrastructure (session_type="deployed").

You can also set a default in the **credentials** notebook (which I have done)

In [2]:
open_session()

<refinitiv.data.session._platform_session.Definition object at 0x1cb36c90e50 {session_name='default-session'}>

### Define callbacks to display the streaming Constituents data
We will use these later to display the streaming data for the Constituent RICs

In [3]:
df = DataFrame()

# Function to initially populate Dataframe, once initial values received for all items
def create_dataframe(streaming_prices):
    global df
    snapshot = streaming_prices.get_snapshot()
    field_names = snapshot.columns[1:]
    instrument_names = snapshot['Instrument'].values
    df = DataFrame(index=instrument_names, columns=field_names)
    for price in streaming_prices:
        for field_name, field_value in price.get_fields().items():
            df.at[price.name, field_name] = field_value
    display(df)

# Function to update dataframe, when we receive updates for individual items
def update_dataframe(streaming_prices, instrument_name, fields):
    global df
    clear_output(wait=True)
    for field_name, field_value in fields.items():
        df.at[instrument_name, field_name] = field_value
    display(df)

### Get the Chain's constituent RICs
Define & snap the current constituents of the Chain and then display them

In [4]:
# Define a Chain object for the Dow Jones Index
dow = pricing.chain.Definition(universe="0#.DJI").get_stream(session=session)
# We just want a snapshot of the current Index
dow.open(with_updates=False)
## Extract the consituent RICs and display
dow_constituents = dow.get_constituents()
print(dow_constituents)

[2021-09-02 15:00:08,071] - [INFO] - [default-session] - [21156] | WebSocket 0 - OMM Protocol - PRICING
Login to websocket wss://apac-3-t3.streaming-pricing-api.refinitiv.com:443/WebSocket successful
['AAPL.OQ', 'AMGN.OQ', 'AXP.N', 'BA.N', 'CAT.N', 'CRM.N', 'CSCO.OQ', 'CVX.N', 'DIS.N', 'DOW.N', 'GS.N', 'HD.N', 'HON.OQ', 'IBM.N', 'INTC.OQ', 'JNJ.N', 'JPM.N', 'KO.N', 'MCD.N', 'MMM.N', 'MRK.N', 'MSFT.OQ', 'NKE.N', 'PG.N', 'TRV.N', 'UNH.N', 'V.N', 'VZ.N', 'WBA.OQ', 'WMT.N']


## Create Streams for the consitutuents
Define streams for all the consituents of the above chain, specifying two fields.   
Specify callbacks to display the field values:
- Call *create_dataframe* once we can received the intial response for all the constituents
- Call *update_dataframe* as and when we receive field update for any of the consituents

In [5]:
# Define our Streaming Price object for all the consituents
pricing_stream = pricing.Definition(universe=dow_constituents, fields=["BID", "ASK"]).get_stream()
# Specify the callback to create and display a dataframe 
# on_complete is invoked once we receive the initial image for all our instruments
pricing_stream.on_complete(create_dataframe)
#Specify the callback to display subsequent updates as and when received
pricing_stream.on_update(update_dataframe)


<refinitiv.data.content.pricing.Stream object at 0x1cb07687b20 {name='['AAPL.OQ', 'AMGN.OQ', 'AXP.N', 'BA.N', 'CAT.N', 'CRM.N', 'CSCO.OQ', 'CVX.N', 'DIS.N', 'DOW.N', 'GS.N', 'HD.N', 'HON.OQ', 'IBM.N', 'INTC.OQ', 'JNJ.N', 'JPM.N', 'KO.N', 'MCD.N', 'MMM.N', 'MRK.N', 'MSFT.OQ', 'NKE.N', 'PG.N', 'TRV.N', 'UNH.N', 'V.N', 'VZ.N', 'WBA.OQ', 'WMT.N']'}>

## Open the streams for all our constituents

In [6]:
# Request all instrument from server and open streams
pricing_stream.open()
# The callbacks we declared earlier should be invoked as and when we receive server responses
# i.e. create_dataframe will be called once we have the initial values for each instrument
# after which, update_dataframe will be invoked as and when any instrument's BID/ASK fields are updated

,BID,ASK
AAPL.OQ,154.2,154.22
AMGN.OQ,224.03,224.27
AXP.N,162.84,162.95
BA.N,218.89,218.97
CAT.N,209.75,209.88
CRM.N,267.64,267.74
CSCO.OQ,59.28,59.29
CVX.N,97.13,97.17
DIS.N,184.71,184.75
DOW.N,62.81,62.83


## Close the streams for the constituents

In [7]:
pricing_stream.close()

,BID,ASK
AAPL.OQ,154.2,154.22
AMGN.OQ,224.03,224.27
AXP.N,162.84,162.95
BA.N,218.89,218.97
CAT.N,209.75,209.88
CRM.N,267.64,267.74
CSCO.OQ,59.28,59.29
CVX.N,97.13,97.17
DIS.N,184.71,184.75
DOW.N,62.81,62.83


<StreamState.Closed: 1>

## Close the session

In [8]:
close_session()